In [1]:
import json
import pickle
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import timeit
#import numpy as np

In [2]:
# load output from hydrating corona_tweets_743
filename = '/Users/jennakline/Downloads/out2.jsonl'
tweet_list = []
with open(filename, 'r') as fopen:
    # each line correspond to a tweet
    for line in fopen:
        tweet_list.append(json.loads(line))

tweet = tweet_list[0]

len(tweet_list) # should output 1,240,043

889879

In [3]:
tweet_list_10k = tweet_list[:10000]
len(tweet_list_10k)

10000

In [4]:
# define methods to get data from json file

def getTweetID(tweet):
    """ If properly included, get the ID of the tweet """
    return tweet.get('id')
    
def getUserIDandScreenName(tweet):
    """ If properly included, get the tweet 
        user ID and Screen Name """
    user = tweet.get('user')
    if user is not None:
        uid = user.get('id')
        screen_name = user.get('screen_name')
        return uid, screen_name
    else:
        return (None, None)

def getRetweetedUserIDandSreenName(tweet):
    """ If properly included, get the retweet 
        source user ID and Screen Name"""
    
    retweet = tweet.get('retweeted_status')
    if retweet is not None:
        return getUserIDandScreenName(retweet)
    else:
        return (None, None)
    
def getRepliedUserIDandScreenName(tweet):
    """ If properly included, get the ID and Screen Name 
        of the user the tweet replies to """
    
    reply_id = tweet.get('in_reply_to_user_id')
    reply_screenname = tweet.get('in_reply_to_screen_name')
    return reply_id, reply_screenname
    
def getUserMentionsIDandScreenName(tweet):
    """ If properly included, return a list of IDs and Screen Names tuple
        of all user mentions, including retweeted and replied users """
        
    mentions = []
    entities = tweet.get('entities')
    if entities is not None:
        user_mentions = entities.get('user_mentions')
        for mention in user_mentions:
            mention_id = mention.get('id')
            screen_name = mention.get('screen_name')
            mentions.append((mention_id, screen_name))
    
    return mentions

    
def getQuotedUserIDandScreenName(tweet):
    """ If properly included, get the ID of the user the tweet is quoting"""
    
    quoted_status = tweet.get('quoted_status')
    
    if quoted_status is not None:
        return getUserIDandScreenName(quoted_status)
    else:
        return (None, None)
    
def getAllInteractions(tweet):
    """ Get all the interactions from this tweet
    
        returns : (tweeter_id, tweeter_screenname), list of (interacting_id, interacting_screenname)
    """
    
    # Get the tweeter
    tweeter = getUserIDandScreenName(tweet)
    
    # Nothing to do if we couldn't get the tweeter
    if tweeter[0] is None:
        return (None, None), []
    
    # a python set is a collection of unique items
    # we use a set to avoid duplicated ids
    interacting_users = set()
    
    # Add person they're replying to
    interacting_users.add(getRepliedUserIDandScreenName(tweet))
    
    # Add person they retweeted
    interacting_users.add(getRetweetedUserIDandSreenName(tweet))
    
    # Add person they quoted
    interacting_users.add(getQuotedUserIDandScreenName(tweet))
    
    # Add mentions
    interacting_users.update(getUserMentionsIDandScreenName(tweet))
  
    # remove the tweeter if he is in the set
    interacting_users.discard(tweeter)
    # remove the None case
    interacting_users.discard((None,None))
    
    # Return our tweeter and their influencers
    return tweeter, list(interacting_users)
    

In [5]:
# create df with tweet id and sentiment analysis

col_names = ['sentiment']
df_tweet_sent = pd.read_csv('/Users/jennakline/Downloads/corona_tweets_743.csv', names = col_names, header = None)

In [10]:
# drop duplicate rows from the tweetid sentiment df
df_tweet_sent2 = df_tweet_sent.reset_index()
df_tweet_sent2 = df_tweet_sent2.drop_duplicates(keep = 'first')
df_tweet_sent2 = df_tweet_sent2.set_index('index')
df_tweet_sent2

,sentiment
index,
1509361182070607873,0.068182
1509361182477484035,0.000000
1509361182641209352,0.000000
1509361182871732224,0.000000
1509361182804643848,0.250000
...,...
1509704625938706438,0.100000
1509704626093891593,0.000000
1509704625938644993,0.025000


In [11]:
topics_list = ["#2019-ncov",
"#2019ncov",
"#chinesevirus",
"#corona",
"#coronaupdate",
"#coronavaccine",
"#coronavaccines",
"#coronavirus",
"#coronawarriors",
"#covid",
"#covid_19",
"#covid-19",
"#covid19",
"#covidiots",
"#faceshield",
"#faceshields",
"#flatteningthecurve",
"#flattenthecurve",
"#frontlineheroes",
"#handsanitizer",
"#health worker",
"#healthworkers",
"#herdimmunity",
"#homeschool",
"#homeschooling",
"#hometasking",
"#kungflu",
"#lockdown",
"#masks4all",
"#n95",
"#ncov",
"#ncov2019",
"#pandemic",
"#pneumonia",
"#ppe",
"#quarantine",
"#sarscov2",
"#selfisolating",
"#socialdistancing",
"#stayathome",
"#stayhome",
"#stayhomestaysafe",
"#vaccine",
"#vaccines",
"#washurhands",
"#washyourhands",
"#wearamask",
"#wfh",
"#workfromhome",
"#workingfromhome",
"#wuhanvirus",
"2019-ncov",
"2019ncov",
"bars closed",
"chinese virus",
"corona",
"corona vaccine",
"corona vaccines",
"coronavirus",
"covid",
"covid_19",
"covid-19",
"covid19",
"covidiots",
"face shield",
"face shields",
"flatten the curve",
"flattening the curve",
"hand sanitizer",
"health worker",
"health workers",
"herd immunity",
"kung flu",
"lockdown",
"n95",
"ncov",
"ncov2019",
"pandemic",
"pneumonia",
"ppe",
"quarantine",
"resturant closed",
"sars cov 2",
"sars cov2",
"sarscov2",
"self isolating",
"social distancing",
"vaccine",
"vaccines",
"wash ur hands",
"wash your hands",
"wear a mask",
"wearamask",
"work from home",
"working from home",
"wuhan virus"]

In [12]:
col_names = ["tweeter_id",
"#2019-ncov",
"#2019ncov",
"#chinesevirus",
"#corona",
"#coronaupdate",
"#coronavaccine",
"#coronavaccines",
"#coronavirus",
"#coronawarriors",
"#covid",
"#covid_19",
"#covid-19",
"#covid19",
"#covidiots",
"#faceshield",
"#faceshields",
"#flatteningthecurve",
"#flattenthecurve",
"#frontlineheroes",
"#handsanitizer",
"#health worker",
"#healthworkers",
"#herdimmunity",
"#homeschool",
"#homeschooling",
"#hometasking",
"#kungflu",
"#lockdown",
"#masks4all",
"#n95",
"#ncov",
"#ncov2019",
"#pandemic",
"#pneumonia",
"#ppe",
"#quarantine",
"#sarscov2",
"#selfisolating",
"#socialdistancing",
"#stayathome",
"#stayhome",
"#stayhomestaysafe",
"#vaccine",
"#vaccines",
"#washurhands",
"#washyourhands",
"#wearamask",
"#wfh",
"#workfromhome",
"#workingfromhome",
"#wuhanvirus",
"2019-ncov",
"2019ncov",
"bars closed",
"chinese virus",
"corona",
"corona vaccine",
"corona vaccines",
"coronavirus",
"covid",
"covid_19",
"covid-19",
"covid19",
"covidiots",
"face shield",
"face shields",
"flatten the curve",
"flattening the curve",
"hand sanitizer",
"health worker",
"health workers",
"herd immunity",
"kung flu",
"lockdown",
"n95",
"ncov",
"ncov2019",
"pandemic",
"pneumonia",
"ppe",
"quarantine",
"resturant closed",
"sars cov 2",
"sars cov2",
"sarscov2",
"self isolating",
"social distancing",
"vaccine",
"vaccines",
"wash ur hands",
"wash your hands",
"wear a mask",
"wearamask",
"work from home",
"working from home",
"wuhan virus"]

In [13]:
# create dict to store # of tweets per user
# Q(i) is the number of posts that user i creates 
# Sij is the number of posts i created and j reposted or tweeted

# initialize dfs to store topics and sentiment
df_topics = pd.DataFrame(columns = col_names)
df_sentiment = pd.DataFrame(columns = col_names)


# loop over all the tweets to get Q and S
Q = {}
s = {}

# get Q
for tweet in tweet_list_10k:
    # find all influencers in the tweet
    tweeter, interactions = getAllInteractions(tweet)
    tweeter_id, tweeter_name = tweeter
    tweet_id = getTweetID(tweet)
    
    print("tweet id: ", tweet_id)
    
    # add up # of tweets per user
    Q[tweeter_id] = Q.setdefault(tweeter_id, 0) + 1
    
    # get full txt of tweet
    txt = tweet['full_text']
    
    S = {}
    
    # get Sij and Q to calculate weights of edges
    for interaction in interactions:
        interact_id, interact_name = interaction
        
        # create dict w/ keys being users j who interacted with tweeter i
        # values is the # of interactions
        S[interact_id] = S.setdefault(interact_id, 0) + 1
        #print("S: ", S)
    
    # append dict S to dict Q as a value
    s[tweeter_id] = s.setdefault(tweeter_id, {})
    s[tweeter_id].update(S)
    #print("Q: ", Q)
    
    for topic in topics_list:
        
        if topic in txt.lower():
            # check to see if tweeter id in df, if not, add
            if tweeter_id not in df_topics.values:
                df_topics.loc[df_topics.shape[0]] = [tweeter_id, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                df_sentiment.loc[df_topics.shape[0]] = [tweeter_id, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
            # create adjacent graph of topics mentioned by users
            df_topics.loc[(df_topics['tweeter_id'] == tweeter_id), topic] = 1
            
            # get existing sentiment for this topic/user combo
            sent1 = df_sentiment.loc[(df_sentiment['tweeter_id'] == tweeter_id), topic]
            sent1 = sent1.iloc[0]
            
            # get sentiment for new tweet from df_tweet_id tbl
            #sent2 = df_tweet_ids.loc[tweet_id][0]
            sent2 = df_tweet_sent2.loc[tweet_id][0]
            
            # add sentiments and store
            sent_total = sent1 + sent2
            df_sentiment.loc[(df_sentiment['tweeter_id'] == tweeter_id), topic] = sent_total
            

tweet id:  1509361182070607873
tweet id:  1509361182477484035
tweet id:  1509361182641209352
tweet id:  1509361182804643848
tweet id:  1509361182871732224
tweet id:  1509361184050499588
tweet id:  1509361185480716290
tweet id:  1509361186080542720
tweet id:  1509361186269286405
tweet id:  1509361187863281665
tweet id:  1509361188152348679
tweet id:  1509361189326925829
tweet id:  1509361189700050944
tweet id:  1509361189943271427
tweet id:  1509361191247773699
tweet id:  1509361191927390217
tweet id:  1509361191944003590
tweet id:  1509361192015261698
tweet id:  1509361192350887937
tweet id:  1509361192460034050
tweet id:  1509361192648781824
tweet id:  1509361192741089284
tweet id:  1509361193261023236
tweet id:  1509361193328205831
tweet id:  1509361193630109696
tweet id:  1509361196943749125
tweet id:  1509361197463879680
tweet id:  1509361198566936580
tweet id:  1509361198998851590
tweet id:  1509361199099568134
tweet id:  1509361199158341634
tweet id:  1509361199242219521
tweet id

tweet id:  1509361329450147847
tweet id:  1509361329555091459
tweet id:  1509361330091925510
tweet id:  1509361330247004168
tweet id:  1509361330867871746
tweet id:  1509361330943250439
tweet id:  1509361331450757127
tweet id:  1509361331522228224
tweet id:  1509361332407070721
tweet id:  1509361332419645442
tweet id:  1509361332671303684
tweet id:  1509361333002768385
tweet id:  1509361333044756480
tweet id:  1509361333778616321
tweet id:  1509361333917339652
tweet id:  1509361334227546115
tweet id:  1509361334256812033
tweet id:  1509361334860894212
tweet id:  1509361335775031300
tweet id:  1509361336026771460
tweet id:  1509361336475693057
tweet id:  1509361337117417478
tweet id:  1509361337218088960
tweet id:  1509361337448775680
tweet id:  1509361337918533638
tweet id:  1509361339294265346
tweet id:  1509361339503886341
tweet id:  1509361340330262533
tweet id:  1509361340351143941
tweet id:  1509361341613899782
tweet id:  1509361341949218822
tweet id:  1509361344348590082
tweet id

tweet id:  1509361469120339976
tweet id:  1509361470856912898
tweet id:  1509361471540584449
tweet id:  1509361471628759047
tweet id:  1509361471838461957
tweet id:  1509361471779704839
tweet id:  1509361472043991042
tweet id:  1509361472060768259
tweet id:  1509361473113444355
tweet id:  1509361473646362626
tweet id:  1509361473679761411
tweet id:  1509361475504254976
tweet id:  1509361475592376321
tweet id:  1509361477471412230
tweet id:  1509361477857136640
tweet id:  1509361477894881280
tweet id:  1509361478108954625
tweet id:  1509361478121439232
tweet id:  1509361478356275202
tweet id:  1509361478490595337
tweet id:  1509361479086129155
tweet id:  1509361479090323462
tweet id:  1509361479342071812
tweet id:  1509361479824248841
tweet id:  1509361480227069960
tweet id:  1509361483636953090
tweet id:  1509361485012561924
tweet id:  1509361485159415808
tweet id:  1509361485394456581
tweet id:  1509361485767659522
tweet id:  1509361485788688394
tweet id:  1509361486124171271
tweet id

tweet id:  1509361595800965121
tweet id:  1509361596518305792
tweet id:  1509361596841308164
tweet id:  1509361600024641537
tweet id:  1509361600729296896
tweet id:  1509361601371123715
tweet id:  1509361601581031425
tweet id:  1509361601706553355
tweet id:  1509361601987686401
tweet id:  1509361602516205568
tweet id:  1509361602570637312
tweet id:  1509361602574880776
tweet id:  1509361602696564736
tweet id:  1509361603061248002
tweet id:  1509361603644313605
tweet id:  1509361603744976896
tweet id:  1509361603984113678
tweet id:  1509361604097626113
tweet id:  1509361604680466437
tweet id:  1509361604869058561
tweet id:  1509361604999389185
tweet id:  1509361605347262464
tweet id:  1509361605586337800
tweet id:  1509361605791793154
tweet id:  1509361606022598656
tweet id:  1509361606194393091
tweet id:  1509361606714658818
tweet id:  1509361607008083972
tweet id:  1509361607159304209
tweet id:  1509361607805227018
tweet id:  1509361609717673987
tweet id:  1509361609902223362
tweet id

tweet id:  1509361736880738312
tweet id:  1509361737811705861
tweet id:  1509361738709442566
tweet id:  1509361739371986945
tweet id:  1509361739464384522
tweet id:  1509361740349382661
tweet id:  1509361740668030976
tweet id:  1509361740785590275
tweet id:  1509361740978470918
tweet id:  1509361741028724736
tweet id:  1509361741045669892
tweet id:  1509361741259493382
tweet id:  1509361741293101057
tweet id:  1509361741888512001
tweet id:  1509361742429753351
tweet id:  1509361743251886081
tweet id:  1509361744237449219
tweet id:  1509361745604796421
tweet id:  1509361745617637386
tweet id:  1509361745671995397
tweet id:  1509361745697308672
tweet id:  1509361746498113539
tweet id:  1509361746527625219
tweet id:  1509361748314263557
tweet id:  1509361748734013440
tweet id:  1509361748943458311
tweet id:  1509361749178343424
tweet id:  1509361750231052290
tweet id:  1509361750566703106
tweet id:  1509361750704984064
tweet id:  1509361751447461889
tweet id:  1509361751489335298
tweet id

...

tweet id:  1509365214273318912
tweet id:  1509365215921680394
tweet id:  1509365215972016134
tweet id:  1509365216211046403
tweet id:  1509365216362082306
tweet id:  1509365217016299521
tweet id:  1509365217624530954
tweet id:  1509365218547466245
tweet id:  1509365219188871171
tweet id:  1509365219474116611
tweet id:  1509365219545403399
tweet id:  1509365220795424770
tweet id:  1509365220937920519
tweet id:  1509365221256794118
tweet id:  1509365221676273666
tweet id:  1509365221952942086
tweet id:  1509365221969862662
tweet id:  1509365222234017807
tweet id:  1509365222884052992
tweet id:  1509365224301895682
tweet id:  1509365224729677824
tweet id:  1509365225023320067
tweet id:  1509365225094533144
tweet id:  1509365225375608842
tweet id:  1509365226679975939
tweet id:  1509365226713534467
tweet id:  1509365227086876672
tweet id:  1509365227544092673
tweet id:  1509365227850276864
tweet id:  1509365228957581320
tweet id:  1509365229586726913
tweet id:  1509365231226609672
tweet id

tweet id:  1509365347819868161
tweet id:  1509365349262802951
tweet id:  1509365349447262209
tweet id:  1509365349522804738
tweet id:  1509365350596554755
tweet id:  1509365350986498051
tweet id:  1509365351586283522
tweet id:  1509365352173514757
tweet id:  1509365352576479232
tweet id:  1509365352622346246
tweet id:  1509365352697794561
tweet id:  1509365353599709185
tweet id:  1509365353918382083
tweet id:  1509365354061041665
tweet id:  1509365354492964875
tweet id:  1509365354790854657
tweet id:  1509365354853588993
tweet id:  1509365355516473345
tweet id:  1509365356728623106
tweet id:  1509365357290659841
tweet id:  1509365357366190084
tweet id:  1509365357718478849
tweet id:  1509365358200717315
tweet id:  1509365358553096195
tweet id:  1509365359001972744
tweet id:  1509365359270240263
tweet id:  1509365359807062020
tweet id:  1509365359811330049
tweet id:  1509365359874383880
tweet id:  1509365360142782469
tweet id:  1509365360197259269
tweet id:  1509365360348344322
tweet id

tweet id:  1509365480150253572
tweet id:  1509365480154353669
tweet id:  1509365480934580224
tweet id:  1509365481370644486
tweet id:  1509365481941049345
tweet id:  1509365481953763333
tweet id:  1509365482381479939
tweet id:  1509365482612314114
tweet id:  1509365483971223554
tweet id:  1509365485053296644
tweet id:  1509365485283921922
tweet id:  1509365486252814341
tweet id:  1509365486319939587
tweet id:  1509365486748049410
tweet id:  1509365488715169795
tweet id:  1509365489000062978
tweet id:  1509365489595830275
tweet id:  1509365490040418305
tweet id:  1509365490409369604
tweet id:  1509365490723995655
tweet id:  1509365490958876673
tweet id:  1509365491760091141
tweet id:  1509365492355846145
tweet id:  1509365492397776896
tweet id:  1509365492561203205
tweet id:  1509365492808663053
tweet id:  1509365493152436224
tweet id:  1509365493634945030
tweet id:  1509365493651722242
tweet id:  1509365493920145409
tweet id:  1509365493970481153
tweet id:  1509365494591094786
tweet id

tweet id:  1509365615018246146
tweet id:  1509365615043264514
tweet id:  1509365615252938755
tweet id:  1509365615450107911
tweet id:  1509365615735181313
tweet id:  1509365615768662022
tweet id:  1509365615978770433
tweet id:  1509365616548884481
tweet id:  1509365617341906949
tweet id:  1509365618080100356
tweet id:  1509365618478252042
tweet id:  1509365619929587715
tweet id:  1509365620286140440
tweet id:  1509365621355548673
tweet id:  1509365621313572873
tweet id:  1509365621460332545
tweet id:  1509365622324740099
tweet id:  1509365622337060873
tweet id:  1509365622546784264
tweet id:  1509365622731378691
tweet id:  1509365623331172358
tweet id:  1509365623822106626
tweet id:  1509365624040214530
tweet id:  1509365624291663875
tweet id:  1509365624874549248
tweet id:  1509365625977679875
tweet id:  1509365626157998090
tweet id:  1509365626309156873
tweet id:  1509365626695221254
tweet id:  1509365626728751107
tweet id:  1509365626736984066
tweet id:  1509365626854379521
tweet id

tweet id:  1509365740402581506
tweet id:  1509365741010632709
tweet id:  1509365741212033024
tweet id:  1509365741547663361
tweet id:  1509365741694652428
tweet id:  1509365742101172230
tweet id:  1509365742277431302
tweet id:  1509365742625599491
tweet id:  1509365743397359616
tweet id:  1509365743741243400
tweet id:  1509365743749640197
tweet id:  1509365744122961925
tweet id:  1509365744240320522
tweet id:  1509365745179783171
tweet id:  1509365745658085377
tweet id:  1509365745830014977
tweet id:  1509365745842819072
tweet id:  1509365746039549954
tweet id:  1509365746186473488
tweet id:  1509365746702458881
tweet id:  1509365747138453508
tweet id:  1509365748015239170
tweet id:  1509365748187156486
tweet id:  1509365749567139841
tweet id:  1509365749596393472
tweet id:  1509365749642579970
tweet id:  1509365750280376320
tweet id:  1509365750515093508
tweet id:  1509365750968066050
tweet id:  1509365750993203204
tweet id:  1509365751676813312
tweet id:  1509365752276652032
tweet id

In [14]:
# BUILD THE NETWORK - optimized approach #

import networkx as nx

# define an empty Directed Graph
# A directed graph is a graph where edges have a direction
# in our case the edges goes from user that sent the tweet to
# the user with whom they interacted (retweeted, mentioned or quoted)
G = nx.DiGraph()

# loop over all the tweets and add edges if the tweet include some interactions

#test = {1473292129774419972, 1066896883363209217}

for tweeter_id in Q:
#for tweeter_id in test:
    # find all influencers in the tweet
    #tweeter, interactions = getAllInteractions(tweet)
    #tweeter_id, tweeter_name = tweeter
    #tweet_id = getTweetID(tweet)
    
    # get Qi for this tweeter
    Qi = Q[tweeter_id]
    #print("tweeter id: ", tweeter_id)
    #print("Qi: ", Qi)
    
    # get Sij for tweeter i
    Si = s[tweeter_id]
    #print("Si: ", Si)
    
    for interact_id in Si:
        Sij = Si[interact_id]
        w = Sij/Qi
        
        G.add_weighted_edges_from([(tweeter_id, interact_id, w)])
        
        # add name as a property to each node
        # with networkX each node is a dictionary
        G.nodes[tweeter_id]['name'] = tweeter_name
        G.nodes[interact_id]['name'] = interact_name
        
nx.write_gpickle(G, "DIGRAPH_743_10k.gpickle") # save graph G in pickle format 

In [16]:
len(G.nodes())

11859

In [17]:
# remove nodes with a low degree
G_copy = G.copy()
remove = [node for node,degree in dict(G_copy.degree()).items() if degree < 2]
G_copy.remove_nodes_from(remove)
len(G_copy.nodes()) 

nx.write_gpickle(G_copy, "DIGRAPH_743_10k_low_deg_removed.gpickle") # save graph G in pickle format 

In [18]:
len(G_copy.nodes()) 

3201

In [57]:
len(remove)

8658

In [22]:
df_topics_low_deg_removed = df_topics.copy(deep=True)
len(df_topics_low_deg_removed)

6011

In [44]:
'1' in df_topics_low_deg_removed.index

False

In [41]:
type(remove)

list

In [58]:
len(df_topics)

6011

In [62]:
df_topics_low_deg_removed = df_topics.copy(deep=True)
df_sentiment_low_deg_removed = df_sentiment.copy(deep=True)

df_topics_low_deg_removed = df_topics_low_deg_removed.set_index('tweeter_id')
df_sentiment_low_deg_removed = df_sentiment_low_deg_removed.set_index('tweeter_id')

for r in remove:
    if r in df_topics_low_deg_removed.index:
        df_topics_low_deg_removed = df_topics_low_deg_removed.drop(r)
    if r in df_sentiment_low_deg_removed.index:
        df_sentiment_low_deg_removed = df_sentiment_low_deg_removed.drop(r)

In [59]:
#df_topics_low_deg_removed = df_topics.copy(deep=True)

for r in remove:
    if r in df_topics_low_deg_removed.index:
        print(r)
        df_topics_low_deg_removed = df_topics_low_deg_removed.drop(r)
        
#len(df_topics_low_deg_removed)

In [63]:
len(df_topics_low_deg_removed.index)

2411

In [64]:
len(df_sentiment_low_deg_removed)

2411

In [66]:
df_sentiment_low_deg_removed

,#2019-ncov,#2019ncov,#chinesevirus,#corona,#coronaupdate,#coronavaccine,#coronavaccines,#coronavirus,#coronawarriors,#covid,...,social distancing,vaccine,vaccines,wash ur hands,wash your hands,wear a mask,wearamask,work from home,working from home,wuhan virus
tweeter_id,,,,,,,,,,,,,,,,,,,,,
1449676288336097281,0,0,0,0.0,0.0,0,0,0.0,0,0.000000,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
37327835,0,0,0,0.0,0.0,0,0,0.0,0,0.000000,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
888399761128267776,0,0,0,0.0,0.0,0,0,0.0,0,0.000000,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
1508261370470359042,0,0,0,0.0,0.0,0,0,0.0,0,0.012103,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
36375459,0,0,0,0.0,0.0,0,0,0.0,0,0.000000,...,0.0,0.05,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504815930957869057,0,0,0,0.0,0.0,0,0,0.0,0,0.000000,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
962923675430264832,0,0,0,0.0,0.0,0,0,0.0,0,0.000000,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
391909630,0,0,0,0.0,0.0,0,0,0.0,0,0.500000,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0


In [67]:
# create TOPIC_USERS_SENTIMENT_DICT_FILE df_sentiment

df_sentiment_2 = df_sentiment_low_deg_removed.copy(deep=True)

# initialize dict using topics_list as keys
topic_users_sent_dict = {key: [] for key in topics_list}

for column in df_sentiment_2:
    #print("column: ", column)
    
    #df_sent_2.loc[df_sentiment2.index, [column]]
    df_sent_2 = pd.DataFrame(df_sentiment_2.loc[df_sentiment_2.index, [column]])
    
    s = {'neut': [], 'pos': [], 'neg':[]}
    
    for index in df_sent_2.index:
        i = df_sent_2[column][index] # i is a df only containing 1 col
        if (i == 1):
            s['neut'].append(index)
            #print("s['neut']: ", s['neut'])
        if (i < 0):
            s['neg'].append(index)
            #print("s['neg']: ", s['neg'])
        if (i > 0):
            s['pos'].append(index)
            #print("s['pos']: ", s['pos'])
                
    topic_users_sent_dict[column].append(s)
    
#print(topic_users_sent_dict)

# topic_users_sent_dict

with open("TOPIC_USERS_SENTIMENT_DICT_FILE_743_10k_low_deg_removed.txt","wb") as f:
    pickle.dump(topic_users_sent_dict, f)

In [69]:
# create TOPIC_USERS_DICT_FILE and BIP_GRAPH_INP from df_topics

df_topics_2 = df_topics_low_deg_removed.copy(deep=True)

# initialize dict using topics_list as keys
topic_users_dict = {key: [] for key in topics_list}
#topic_users_dict['#2019-ncov'].append(1)
#topic_users_dict

for column in df_topics_2:
    for index in df_topics_2.index:
        i = df_topics_2[column][index]
        if (i == 1):
            topic_users_dict[column].append(index)
            
topic_users_dict

with open("TOPIC_USERS_DICT_FILE_743_10k_low_deg_removed.txt","wb") as f:
    pickle.dump(topic_users_dict, f)

# create bipartite graph of users and topics
# create bipartite graph
G_BIP = nx.to_networkx_graph(topic_users_dict)

# validate is biparite
bipartite.is_bipartite(G_BIP)

nx.write_gpickle(G_BIP, "BIP_GRAPH_INP_743_10k_low_deg_removed.gpickle")
nx.write_adjlist(G_BIP, "BIP_GRAPH_INP_743_10k_low_deg_removed.adjlist")

In [70]:
nx.is_bipartite(G_BIP)

True

In [72]:
len(G_BIP.nodes())

2507

In [73]:
G_BIP.nodes()

NodeView(('#2019-ncov', '#2019ncov', '#chinesevirus', '#corona', '#coronaupdate', '#coronavaccine', '#coronavaccines', '#coronavirus', '#coronawarriors', '#covid', '#covid_19', '#covid-19', '#covid19', '#covidiots', '#faceshield', '#faceshields', '#flatteningthecurve', '#flattenthecurve', '#frontlineheroes', '#handsanitizer', '#health worker', '#healthworkers', '#herdimmunity', '#homeschool', '#homeschooling', '#hometasking', '#kungflu', '#lockdown', '#masks4all', '#n95', '#ncov', '#ncov2019', '#pandemic', '#pneumonia', '#ppe', '#quarantine', '#sarscov2', '#selfisolating', '#socialdistancing', '#stayathome', '#stayhome', '#stayhomestaysafe', '#vaccine', '#vaccines', '#washurhands', '#washyourhands', '#wearamask', '#wfh', '#workfromhome', '#workingfromhome', '#wuhanvirus', '2019-ncov', '2019ncov', 'bars closed', 'chinese virus', 'corona', 'corona vaccine', 'corona vaccines', 'coronavirus', 'covid', 'covid_19', 'covid-19', 'covid19', 'covidiots', 'face shield', 'face shields', 'flatten t